# Angular focus stacking experiment

In [60]:
from __future__ import print_function, division
import numpy as np
import collections as co

# constants
pi = np.pi

# trig functions
sind = lambda x : np.sin(np.deg2rad(x))
cosd = lambda x : np.cos(np.deg2rad(x))
tand = lambda x : np.tan(np.deg2rad(x))
arcsind = lambda x : np.rad2deg(np.arcsin(x))
arctand = lambda x : np.rad2deg(np.arctan(x))

In [10]:
%precision 2

u'%.2f'

## Test COP shift compensation

In [109]:
# cop compensation  

def get_COP_shift_compensation(alpha, hy, hz):
    """returns the amount of compensation required to restore the COP following
    a rotation of the lens about a pivot point away from the COP
    
    Parameters
    ----------
    alpha : real
        angle of rotation of the lens about x-axis in degrees
    hy : real
        y component of the distance of the COP from the pivot 
    hz : real 
        z component of the distance of the COP from the pivot
    """
    shiftYZ = co.namedtuple('comp', ['delY', 'delZ'])
    # shiftY, shiftZ are the shift of the COP along the y and z directions
    shiftY = -hz*sind(alpha) + hy*(1 - cosd(alpha))
    shiftZ = -hy*sind(alpha) - hz*(1 - cosd(alpha))
    return shiftYZ(-shiftY, -shiftZ)

In [222]:

alphas = [15.0, 10.0, 5.0, 0, -5.0, -10.0, -15.0]

hz = 32
hy = -10
for alpha in alphas:
    delyz = get_COP_shift_compensation(alpha=alpha, hy=hy, hz=hz)
    print('alpha = {:2.1f}, dely = {:2.2f}, delz = {:2.2f}'.format(alpha, delyz.delY, delyz.delZ))

alpha = 15.0, dely = 8.62, delz = -1.50
alpha = 10.0, dely = 5.71, delz = -1.25
alpha = 5.0, dely = 2.83, delz = -0.75
alpha = 0.0, dely = 0.00, delz = -0.00
alpha = -5.0, dely = -2.75, delz = 0.99
alpha = -10.0, dely = -5.40, delz = 2.22
alpha = -15.0, dely = -7.94, delz = 3.68


## Focusing on tilted plane

In [61]:
#  helper f
def compute_zoDash(zo, f, mp, d, alpha):
    """formula for computing the image plane distance from the pivot (origin of {C}) if
    lens is tilted about the ENPP. That is de=0, deDash = d
    
    @param: zo: object distance along z-axis
    @param: f: focal length
    @param: mp: pupil magnification
    @param: d: distance from ENPP to EXPP
    @param: alpha: angle of image plane tilt about x-axis (in degrees)
    """
    c = cosd(alpha)
    s = sind(alpha)
    return d*c + mp*f*zo*(mp*c**2 + s**2)/(zo*mp*c + f)

def compute_beta(zo, f, mp, alpha):
    """formula for computing the object plane tilt angle if
    lens is tilted about the ENPP. That is de=0, deDash = d
    
    @param: zo: object distance along z-axis
    @param: f: focal length
    @param: mp: pupil magnification
    @param: alpha: angle of image plane tilt about x-axis (in degrees)
    """
    c = cosd(alpha)
    s = sind(alpha)
    t = -s*(mp*zo + (1-mp)*f*c)/(f*(mp*c**2 + s**2))
    return arctand(t)

In [62]:
# Iterative algorithm to find alpha, given beta

def g_of_alpha_is_monotonic(zo, f, mp):
    """test (method 2) to see if first derivative has real roots
    
    Note:
    -----
    The test for monotonicity using D is only relevant if mp < 1.
    """
    if mp == 1:
        D = 1  # the equation is linear
    else:
        a = 1
        b = f*(1 + mp)/(mp*zo)
        c = -(1 - 2*mp)/(1 - mp)
        d = -f/(mp*zo)
        Q = c/(3.0*a) - b**2/(9.0*a**2)
        R = -b**3/(27.0*a**3) + b*c/(6.0*a**2) - d/(2.0*a)
        D = Q**3 + R**2
    monotonicity = D > 0 if mp < 1 else True
    return monotonicity, D

def alpha_thin(zo, f, beta):
    return -arcsind(f*tand(beta)/zo)

def get_step(err):
    return 0.5*err

def estimate_alpha(zo, f, mp, beta, maxSteps=250, maxError=1e-16, retSteps=False):
    """iterative method to estimate alpha given beta and other parameters
    """
    assert g_of_alpha_is_monotonic(zo, f, mp)[0], 'g(alpha) is not monotonic'
    i = 0
    t = tand(beta)
    alphaInit = alpha_thin(zo=zo, f=f, beta=beta)
    s = sind(alphaInit)
    c = cosd(alphaInit)
    err = t + s*(mp*zo + f*(1-mp)*c)/(f*(mp*c**2 + s**2)) # Initial merit function value
    prevErr = 0
    alpha_i = alphaInit
    deltaStep = get_step(err)  # deltaErr = err - prevErr = err
    if retSteps:
        alphas, errors, steps = [], [], []
        alphas.append(alphaInit)
        errors.append(err)
        steps.append(deltaStep)
    # begin iteration
    while (abs(err) > maxError) and (i < maxSteps):
        i += 1
        alpha_i += deltaStep
        prevErr = err
        s = sind(alpha_i)
        c = cosd(alpha_i)
        err = t + s*(mp*zo + f*(1-mp)*c)/(f*(mp*c**2 + s**2))
        if err*prevErr < 0:  # if error changes sign
            deltaErr = err - prevErr
            deltaStep = get_step(deltaErr)
        if retSteps:
            alphas.append(alpha_i)
            errors.append(err)
            steps.append(deltaStep)
    # end of iteration ----
    if retSteps:
        return alpha_i, i, err, alphas, errors, steps
    else:
        return alpha_i, i, err

In [92]:
# Optical system
mp = 1.0 
f = 180.0
zo = -4038.6  # 159 inch 

### finding $d$ 

The idea is to focus on the three planes without tilting the lens, and measure the image plane distance from the COP (already located), 

In [187]:
# observed zoDash was about 180 mm
d = -7.25
compute_zoDash(-4648.2, 180.0, 1.0, d, 0)

180.00

In [189]:
# observed zoDash was about 181 mm
d = -7.4
compute_zoDash(-4038.6, 180.0, 1.0, d, 0)

181.00

In [201]:
# observed zoDash was about 183 mm
d = -6.97
compute_zoDash(-3429.0, 180.0, 1.0, d, 0)

183.00

In [218]:
# observed zoDash was about 195 mm
d = -8.53
compute_zoDash(-1557.02, 180.0, 1.0, d, 0)

195.00

In [219]:
(-8.53 -6.97 -7.4 -7.25)/4.0

-7.54

We assume d =-8.0 mm

<del>So, we will use $\pm 19^o$ tilt of the lens.<\del>

Have to use a different strategy

In [124]:
# compensation required for tilting the lens
#alphas = [-19, 19]

alphas = [-5, 5]

hz = 32
hy = -10
for alpha in alphas:
    delyz = get_COP_shift_compensation(alpha=alpha, hy=hy, hz=hz)
    print('alpha = {:2.1f}, dely = {:2.2f}, delz = {:2.2f}'.format(alpha, delyz.delY, delyz.delZ))


alpha = -5.0, dely = -2.75, delz = 0.99
alpha = 5.0, dely = 2.83, delz = -0.75


In [221]:
print('alpha =', -5)
zoOffset = -5000
zoNew = zo+zoOffset
beta = compute_beta(zoNew , f, mp, alpha=-5)
betaDash = 90 - beta
print('beta =', beta)
print('betaDash =', betaDash)
print('new zo =', zoNew, 'mm')
print('new zo =', zoNew*0.00328084, 'foot')

alpha = -5
beta = -77.1291994174
betaDash = 167.129199417
new zo = -9038.6 mm
new zo = -29.654200424 foot


In [220]:
d = -8.0
compute_zoDash(zoNew, f, mp, d, alpha=-5)

176.40

In [223]:
alphas = [-5.0,]

hz = 32
hy = -10
for alpha in alphas:
    delyz = get_COP_shift_compensation(alpha=alpha, hy=hy, hz=hz)
    print('alpha = {:2.1f}, dely = {:2.2f}, delz = {:2.2f}'.format(alpha, delyz.delY, delyz.delZ))

alpha = -5.0, dely = -2.75, delz = 0.99


##### Date: 11-12-2016

We will use $\pm 19^o$ tilt of the lens, with $z_o$ being the middle cutout

In [226]:
mp = 1.0 
d = -8.0
f = 180.0
zo = -4038.6  # 159 inch 

In [229]:
compute_beta(zo, f, mp, alpha=-19)

-82.20

In [234]:
# compensation required for tilting the lens
alphas = [-19, -18, -17, -16]

hz = 32
hy = -10
for alpha in alphas:
    delyz = get_COP_shift_compensation(alpha=alpha, hy=hy, hz=hz)
    print('alpha = {:2.1f}, dely = {:2.2f}, delz = {:2.2f}'.format(alpha, delyz.delY, delyz.delZ))


alpha = -19.0, dely = -9.87, delz = 5.00
alpha = -18.0, dely = -9.40, delz = 4.66
alpha = -17.0, dely = -8.92, delz = 4.32
alpha = -16.0, dely = -8.43, delz = 4.00


In [237]:
alphas = [-16, -16.5, -17, -17.5, -18, -18.5, -19]

hz = 32
hy = -10
for alpha in alphas:
    delyz = get_COP_shift_compensation(alpha=alpha, hy=hy, hz=hz)
    print('alpha = {:2.1f}, dely = {:2.2f}, delz = {:2.2f}'.format(alpha, delyz.delY, delyz.delZ))

alpha = -16.0, dely = -8.43, delz = 4.00
alpha = -16.5, dely = -8.68, delz = 4.16
alpha = -17.0, dely = -8.92, delz = 4.32
alpha = -17.5, dely = -9.16, delz = 4.49
alpha = -18.0, dely = -9.40, delz = 4.66
alpha = -18.5, dely = -9.64, delz = 4.83
alpha = -19.0, dely = -9.87, delz = 5.00
